In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/02 14:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet

--2022-11-24 09:33:11--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:21d6:1000:b:20a5:b140:21, 2600:9000:21d6:800:b:20a5:b140:21, 2600:9000:21d6:d800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:21d6:1000:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302633211 (289M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-02.parquet’

fhvhv_tripdata_2021 100%[===================>] 288,61M  45,9MB/s    in 6,7s    

2022-11-24 09:33:19 (43,2 MB/s) - ‘fhvhv_tripdata_2021-02.parquet’ saved [302633211/302633211]



In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-02.parquet')

In [5]:
df = df.repartition(24)

In [6]:
df.write.parquet('fhvhv/2021/02/')

22/11/24 09:37:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


22/11/24 09:37:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [14]:
# What's the size of the folder with results (in MB)?
import os

# assign size
size = 0
 
# assign folder path
Folderpath = 'fhvhv/2021/02/'
 
# get size
for ele in os.scandir(Folderpath):
    size+=os.path.getsize(ele)
     
print(f"Size: {size/(1024*1024)} MB")

print(f"Original size: {os.stat('fhvhv_tripdata_2021-02.parquet').st_size/(1024*1024)}")

# https://stackoverflow.com/questions/54218006/why-does-the-repartition-method-increase-file-size-on-disk

Size: 515.6801538467407 MB
Original size: 288.6135206222534


In [5]:
df = spark.read.parquet('fhvhv/2021/02/')

In [6]:
from pyspark.sql import functions as F

### Question 3. Count records 

In [8]:
# How many taxi trips were there on February 15?
df.filter(F.to_date(df.pickup_datetime) == '2021-02-15').count()

367170

### Question 4. Longest trip for each day

In [7]:
# Trip starting on which day was the longest? 
df.registerTempTable("fhvhv_tripdata_2021_02")

/usr/local/Cellar/apache-spark/3.3.1/libexec/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
# SQL version
df_longest_trip_for_each_day = spark.sql("""
SELECT 
    DATE_TRUNC('day',pickup_datetime) as day,
    max(DATEDIFF(second, pickup_datetime, dropoff_datetime)) AS difference
FROM
    fhvhv_tripdata_2021_02
GROUP BY 1
ORDER BY 2 DESC

""")
df_longest_trip_for_each_day.show()

### Question 5. Most frequent `dispatching_base_num`

In [ ]:
# SQL version
df_most_frequent_dispatching_base_num = spark.sql("""
SELECT 
    dispatching_base_num,
    count(dispatching_base_num) as n_dispatching_base_num
FROM
    fhvhv_tripdata_2021_02
GROUP BY 1
ORDER BY 2 DESC
""")
df_most_frequent_dispatching_base_num.show()

In [25]:
# PySpark version
df.groupBy('dispatching_base_num').count().orderBy('count', ascending=False).limit(5).show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+



**Both version have 1 stage**

### Question 6. Most common locations pair

In [37]:
# Read Zones data
df = spark.read.parquet('code/zones')
# Create temp table
df.createOrReplaceTempView("zone")
df.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [48]:
df_most_common_locations_pair = spark.sql("""
SELECT 
    pu.Zone,
    do.Zone,
    most_common_pair.location_pair_count
FROM
(SELECT 
    PuLocationID,
    DOLocationID,
    count(*) as location_pair_count
FROM
    fhvhv_tripdata_2021_02
GROUP BY 1, 2
Order by 3 DESC
limit 1) as most_common_pair 
LEFT OUTER JOIN zone as pu ON pu.LocationID=most_common_pair.PuLocationID
LEFT OUTER JOIN zone as do ON do.LocationID=most_common_pair.DOLocationID
""")
df_most_common_locations_pair.show()

+-------------+-------------+-------------------+
|         Zone|         Zone|location_pair_count|
+-------------+-------------+-------------------+
|East New York|East New York|              45041|
+-------------+-------------+-------------------+

